## SMAVA

Test exercise - Question 2

Try a logistic regressor on just a subset.

In [1]:
## weaves
## smava

getwd()

## load in packages
library(caret)
library(ranger)
library(tidyverse)
library(e1071)
library(MASS)
library(rpart)
library(rpart.plot)
library(ipred)
library(mlbench)
library(pROC)
library(gbm)
library(dplyr)

library(doMC)

registerDoMC(cores = 4)

options(useFancyQuotes = TRUE)

[1] "/misc/build/0/pyeg0/caret/classr1"

Loading required package: lattice

Loading required package: ggplot2

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v tibble  2.1.3     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0
v purrr   0.3.3     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
x purrr::lift()   masks caret::lift()


Attaching package: 'MASS'


The following object is masked from 'package:dplyr':

    select


Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var


Loaded gbm 2.1.5

Loading required package: foreach


Attaching package: 'foreach'


The following objects are masked from 'package:purrr':

    accumulate, when


Loading required package: iterators

Loading required package: parallel



In [2]:
load("bak/in/train.rdata")
load("bak/in/test.rdata")

train0 <- data.frame(train) # a local copy.

In [3]:
fit1 <- train(
  form = accepted ~ x1 + x10 + x4 + bank,
  data = train,
  trControl = trainControl(method = "cv", number = 5),
  method = "glm",
  family = "binomial"
)

In [4]:
fit1

Generalized Linear Model 

134969 samples
     4 predictor
     2 classes: 'NO', 'YES' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 107975, 107975, 107975, 107975, 107976 
Resampling results:

  Accuracy   Kappa    
  0.5995451  0.1977802


In [6]:
## Not very good. Let's try by bank.
smava1 <- list()
smava1$banks <- unique(train[["bank"]])
smava1$banks

[1] B14 B4  B25 B3  B1  B8  B18 B21 B6  B15 B16 B2  B12 B27 B7  B30 B9  B22 B28
[20] B17 B26 B23 B11 B20 B19 B13 B29 B24 B5  B10
30 Levels: B1 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B2 B20 B21 B22 ... B9

In [8]:
tag <- smava1$banks[1]
tag
colnames(train)

[1] B14
30 Levels: B1 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B2 B20 B21 B22 ... B9

[1] "customerNumber" "bank"           "x1"             "x2"            
 [5] "x3"             "x4"             "x5"             "x6"            
 [9] "x7"             "x8"             "x9"             "x10"           
[13] "accepted"       "interestRate"

In [9]:
cols <- colnames(train)
cols <- cols[grep('^x', cols)]
cols <- setdiff(cols, "x3")
cols <- c(cols, "accepted")
cols

[1] "x1"       "x2"       "x4"       "x5"       "x6"       "x7"      
 [7] "x8"       "x9"       "x10"      "accepted"

In [10]:
train1 <- train[ train$bank == tag, cols]

train1$x2na <- 0
train1[is.na(train1$x2), "x2na"] <- 1

v0 <- train1[train1$x2 & train1$x2na == 0, "x2"]
v0 <- mean(v0[["x2"]])

train1[is.na(train1$x2), "x2"] <- v0

head(train1)

# test1[ is.na(test1$x2), "x2" ] <- smava0$x2impute

x1,x2,x4,x5,x6,x7,x8,x9,x10,accepted,x2na
<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
2.6820549,11.1005105,4.026472,-1.487079,FBW,M98CB,2.419460,0.4615068,0.2702748,NO,0
0.9196841,1.0918998,9.150705,-1.307155,ACS,IQ4NP,7.853798,0.9472202,0.6733410,NO,0
0.1525603,0.8607999,-5.818878,4.331156,ACS,GPX9G,-1.783852,0.3560776,0.4760840,YES,0
4.9194948,8.2227958,0.668966,5.256914,FBW,IQ4NP,4.854271,0.1086605,0.4035353,NO,1
0.3707626,0.1313585,3.866158,-1.419184,ACS,IQ4NP,3.200393,0.5127080,0.4306942,YES,0
0.2420967,1.3637400,1.624817,4.140840,ACS,PKA0F,6.125612,0.2527484,0.5465620,YES,0


In [ ]:
fit1x <- train(
  form = accepted ~ .,
  data = train1,
  trControl = trainControl(method = "cv", number = 3),
  method = "glm",
  family = "binomial"
)
fit1x